# Codebase

## Imports

In [1]:
import sys
import time
import paramiko
 
# Python Image Library
from PIL import Image #works only in local env

from naoqi import ALProxy

/usr/local/lib/python2.7/site-packages/paramiko/transport.py:33: CryptographyDeprecationWarning: Python 2 is no longer supported by the Python core team. Support for it is now deprecated in cryptography, and will be removed in the next release.
  from cryptography.hazmat.backends import default_backend


## Settings

In [2]:
NAOIP = '192.168.0.241'
PORT = 9559
NAME = "nao"
passwd = "19981"

## TEST
tts = 'ALTextToSpeech'
text = ALProxy(tts, NAOIP, PORT)
text.say("Hi NAOsolutions, thanks that you guys develop me further!")

In [3]:
!python --version

Python 2.7.18


## SSH

In [29]:
paramiko.util.log_to_file("paramiko.log")

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(NAOIP, username=NAME, password=passwd)
#######
print("Connected....")
#######
ssh.close()
print("Connection ended...")

Connected....
Connection ended...


## picture

### video module

In [6]:
def showNaoImage(IP, PORT,camera,resolution, colorSpace):
  """
  First get an image from Nao, then show it on the screen with PIL.
  """
  camProxy = ALProxy("ALVideoDevice", IP, PORT)
  videoClient = camProxy.subscribeCamera("python_client",0, resolution, colorSpace, 5)
  t0 = time.time()

  # Get a camera image.
  # image[6] contains the image data passed as an array of ASCII chars.
  naoImage = camProxy.getImageRemote(videoClient)
  t1 = time.time()
  
  # Time the image transfer.
  print "acquisition delay ", t1 - t0
  camProxy.unsubscribe(videoClient)

  # Now we work with the image returned and save it as a PNG  using ImageDraw
  # package.

 
  # Get the image size and pixel array.
  imageWidth = naoImage[0]
  imageHeight = naoImage[1]
  array = naoImage[6]


  # Create a PIL Image from our pixel array.
  im = Image.frombytes("RGB", (imageWidth, imageHeight), array)

  # Save the image.
  im.save("camImage.png", "PNG")
  im.show()


camera = 1 # 0 = top camera, 1 = bottom camera

resolution = 3 # 0 = QQVGA, 1 = QVGA, 2 = VGA

colorSpace = 11 # http://doc.aldebaran.com/2-5/family/robots/video_robot.html#cameracolorspace-mt9m114

naoImage = showNaoImage(NAOIP, PORT,camera,resolution, colorSpace)

acquisition delay  1.45231795311


### without function

In [7]:
photoCaptureProxy = ALProxy("ALPhotoCapture", NAOIP, PORT)

In [8]:
## takes 3 pictures
photoCaptureProxy.setResolution(2)
photoCaptureProxy.setPictureFormat("jpg")
photoCaptureProxy = photoCaptureProxy.takePictures(3, "/home/nao/recordings/cameras/", "image") #3 picturess

## Speech recognization

### In Built

In [12]:
speechProxy = ALProxy("ALSpeechRecognition", NAOIP, PORT)
speechProxy.setLanguage("English")
#######
vocabulary = ["yes", "no", "please"]
speechProxy.setVocabulary(vocabulary, False)
#######
speechProxy.subscribe("Test_ASR")
print 'Speech recognition engine started'
time.sleep(20)
speechProxy.unsubscribe("Test_ASR")

### Google Speech

In [1]:
from google.cloud import speech_v1 as speech

/usr/local/lib/python2.7/site-packages/google/auth/crypt/_cryptography_rsa.py:22: CryptographyDeprecationWarning: Python 2 is no longer supported by the Python core team. Support for it is now deprecated in cryptography, and will be removed in the next release.
  import cryptography.exceptions


In [13]:
def speech_to_text(config, audio):
    client = speech.SpeechClient.from_service_account_file("key_for_google.json")
    response = client.recognize(config=config, audio=audio)
    print_sentences(response)


def print_sentences(response):
    for result in response.results:
        best_alternative = result.alternatives[0]
        transcript = best_alternative.transcript
        confidence = best_alternative.confidence
        print "-" * 80
        print "Transcript:", transcript
        print "Confidence:", confidence*100,"%"


config = dict(language_code="en-US")
audio = dict(uri="gs://cloud-samples-data/speech/brooklyn_bridge.flac")

In [14]:
speech_to_text(config, audio)

--------------------------------------------------------------------------------
Transcript: how old is the Brooklyn Bridge
Confidence: 98.2160568237 %


### SpeechRecognition

In [2]:
import speech_recognition as sr

#### with audio file

In [6]:
r = sr.Recognizer()

with sr.AudioFile('test_audio.wav') as source:
    audio_text = r.listen(source)
    try:
        # using google speech recognition
        text = r.recognize_google(audio_text)
        print('Converting audio transcripts into text ...')
        print(text)
    except:
            print('Sorry.. run again...')

Converting audio transcripts into text ...
what if somebody decides to break it be careful that you keep adequate coverage but look for places to save money baby it's taking longer to get things squared away than the bankers expected during the wife for once company may win her tax hated retirement income as helpful as our cost on the two naked bone when the title of this type of song is in question there's no dying or waxing or gassing needed maybe personalized leather hard at work on a flat surface and smooth out the simplest kind of separate system uses a single self-contained unit the old shop at it still holds a good mechanic is usually a bad boss both figures would go higher in later years cabinets chest doll houses at set


#### with mic:

In [7]:
with sr.Microphone() as source:
    print("Talk")
    audio_text = r.listen(source)
    print("Time over, thanks")

AttributeError: Could not find PyAudio; check installation

In [ ]:
def speech_recognize(time):
    with sr.Microphone() as source:
        self.r.adjust_for_ambient_noise(source)
        print 'mic on... start recording!'
        audio = self.r.listen(source, phrase_time_limit=time)
        try:
            print 'Reconized words: ' + self.r.recognize_google(audio)
            return self.r.recognize_google(audio)
        except LookupError, e:
            print e
            return 'there was an error!'
        except sr.UnknownValueError, e:
            print e
            print 'what do you mean? i do not understand'
            return ''